In [12]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


train_dir = 'dataset/cars/train'
test_dir = 'dataset/cars/validation'

# Doing data augmentation and loading the training images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize the data
    rotation_range=30,  # Rotate images by upto 30 degrees
    width_shift_range=0.2,  # Shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Shift images vertically by 20% of the height
    shear_range=0.2,  # Apply random shear transformations
    zoom_range=0.2,  # Zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Step 3: Load and split the dataset using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize the images to match VGG16 input size
    batch_size=32,
    class_mode="categorical",
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,  # Keep the order of images for evaluation
)

# Step 4: Load pre-trained VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Step 5: Add custom classification layers on top of VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Step 6: Fine-tune the pre-trained layers
for layer in base_model.layers[:15]:
    layer.trainable = False
for layer in base_model.layers[15:]:
    layer.trainable = True

# Step 7: Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Step 8: Train the model using the generator
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(test_generator),
)

# Step 9: Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Found 3346 images belonging to 10 classes.
Found 357 images belonging to 10 classes.
Epoch 1/15
105/105 [==============================] - 221s 2s/step - loss: 2.0114 - accuracy: 0.2878 - val_loss: 1.7972 - val_accuracy: 0.3389
Epoch 2/15
105/105 [==============================] - 219s 2s/step - loss: 1.6359 - accuracy: 0.4235 - val_loss: 1.5211 - val_accuracy: 0.4538
Epoch 3/15
105/105 [==============================] - 221s 2s/step - loss: 1.3026 - accuracy: 0.5472 - val_loss: 1.3804 - val_accuracy: 0.5154
Epoch 4/15
105/105 [==============================] - 858s 8s/step - loss: 1.0804 - accuracy: 0.6339 - val_loss: 1.1164 - val_accuracy: 0.6078
Epoch 5/15
105/105 [==============================] - 223s 2s/step - loss: 0.8760 - accuracy: 0.7023 - val_loss: 1.2233 - val_accuracy: 0.6078
Epoch 6/15
105/105 [==============================] - 224s 2s/step - loss: 0.7507 - accuracy: 0.7397 - val_loss: 1.0302 - val_accuracy: 0.6527
Epoch 7/15
105/105 [==============================] - 224

In [13]:
model.save('car_make_model.h5')